### CREATE A FUNCTION TO EVALUATE THE MODEL BEFORE SENDING IT ONLINE


In [10]:
# from torch.utils.data import DataLoader
# import sys
# import os 
# # Add the project root directory to Python path
# sys.path.append(os.path.dirname(os.getcwd()))
# from data.dataset import Dataset
# from models.dinov2_advanced import DinoV2Finetune_advanced
# import hydra







In [ ]:
# #create the dataset with the test set
# model=DinoV2Finetune_advanced()
# checkpoint_path=[
#     "/users/eleves-b/2023/sylvain.dehayem-kenfouo/projet_final_modal/checkpoints/dinov2_advanced_2025-05-04_04-11-10_with_Gelu.pt",
#     "/users/eleves-b/2023/sylvain.dehayem-kenfouo/projet_final_modal/checkpoints/dinov2_advanced_2025-05-04_03-20-35_with_Gelu.pt", 
#     "/users/eleves-b/2023/sylvain.dehayem-kenfouo/projet_final_modal/checkpoints/dinov2_advanced_2025-05-04_02-09-07_with_Gelu.pt", 
#     "/users/eleves-b/2023/sylvain.dehayem-kenfouo/projet_final_modal/checkpoints/dinov2_advanced_2025-05-03_06-12-30_last.pt"

# ]
# def run_evaluation(model , checkpoint_path):
#     @hydra.main(config_path="configs", config_name="train")
#     def evaluate_model(cfg):
#         test_dataset=Dataset(cfg.datamodule.dataset_path, "test", transforms=hydra.utils.instantiate(cfg.datamodule.test_transform), metadata=cfg.datamodule.metadata)
#         test_loader=DataLoader(test_dataset, batch_size=cfg.datamodule.batch_size, shuffle=False, num_workers=cfg.datamodule.num_workers)
#         loss_fn = hydra.utils.instantiate(cfg.loss_fn)
#         model.load_state_dict(checkpoint)
#         print("Model loaded")
#         for _, batch in enumerate(test_loader):
#             batch["image"] = batch["image"].to(device)
#             batch["target"] = batch["target"].to(device).squeeze()
#             with torch.no_grad():
#                 preds = model(batch).squeeze().cpu().numpy()
#             loss = loss_fn(preds, batch["target"])
#             epoch_val_loss += loss.detach().cpu().numpy() * len(batch["image"])
#             num_samples_val += len(batch["image"])
#         epoch_val_loss /= num_samples_val
#         val_metrics["val/loss_epoch"] = epoch_val_loss
#         logger_std.info(f"----------------Epoch {epoch} val loss: {epoch_val_loss:.4f}----------------")
#     evaluate_model()



Using cache found in /users/eleves-b/2023/sylvain.dehayem-kenfouo/.cache/torch/hub/facebookresearch_dinov2_main


In [21]:
# for checkpoint in checkpoint_path:
#     print("--------------------------------\n")
#     print(f"Evaluating model with checkpoint: {checkpoint} \n")
#     run_evaluation(model, checkpoint)


--------------------------------

Evaluating model with checkpoint: /users/eleves-b/2023/sylvain.dehayem-kenfouo/projet_final_modal/checkpoints/dinov2_advanced_2025-05-04_04-11-10_with_Gelu.pt 



/tmp/ipykernel_4066991/1543475991.py:11: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="configs", config_name="train")
usage: ipykernel_launcher.py [--help] [--hydra-help] [--version]
                             [--cfg {job,hydra,all}] [--resolve]
                             [--package PACKAGE] [--run] [--multirun]
                             [--shell-completion] [--config-path CONFIG_PATH]
                             [--config-name CONFIG_NAME]
                             [--config-dir CONFIG_DIR]
                             [--experimental-rerun EXPERIMENTAL_RERUN]
                             [--info [{all,config,defaults,defaults-tree,plugins,searchpath}]]
                             [overrides ...]
ipykernel_launcher.py: error: unrecognized arguments: --f=/users/eleves-b/2023/sylvain.dehayem-kenfouo/.local/share/jupyter/runtime/kernel-v3f4ae586a

SystemExit: 2

/users/eleves-b/2023/sylvain.dehayem-kenfouo/miniconda/envs/challenge/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [16]:
import torch
import torch.nn as nn


class DinoV2Finetune_large(nn.Module):
    def __init__(self, frozen=False):
        super().__init__()
        self.backbone =torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14_reg')
        self.backbone.head = nn.Identity()
        self.dim = self.backbone.norm.normalized_shape[0]
        if frozen:
            for param in self.backbone.parameters():
                param.requires_grad = False
        self.regression_head = nn.Sequential(
            nn.Linear(self.backbone.norm.normalized_shape[0], 1),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.backbone(x["image"])
        x = self.regression_head(x)
        return x
class DinoV2Finetune_small(nn.Module):
    def __init__(self, frozen=False):
        super().__init__()
        self.backbone =torch.hub.load("facebookresearch/dinov2", "dinov2_vitb14_reg")
        self.backbone.head = nn.Identity()
        self.dim = self.backbone.norm.normalized_shape[0]
        if frozen:
            for param in self.backbone.parameters():
                param.requires_grad = False
        self.regression_head = nn.Sequential(
            nn.Linear(self.backbone.norm.normalized_shape[0], 1),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.backbone(x["image"])
        x = self.regression_head(x)
        return x

In [17]:
large_dinov= DinoV2Finetune_large()
small_dinov= DinoV2Finetune_small()
#check parameters of the model
# print(model.parameters())


Using cache found in /users/eleves-b/2023/sylvain.dehayem-kenfouo/.cache/torch/hub/facebookresearch_dinov2_main
Using cache found in /users/eleves-b/2023/sylvain.dehayem-kenfouo/.cache/torch/hub/facebookresearch_dinov2_main


In [14]:
import torch
import torch.nn as nn
from torchinfo import summary


def print_model_info(model):
    """Print detailed information about the model"""
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Frozen parameters: {total_params - trainable_params:,}")
    
    # Print model architecture
    print("\nModel Architecture:")
    print(model)
    
    # Print parameter details for each layer
    print("\nDetailed Layer Information:")
    for name, param in model.named_parameters():
        print(f"{name}: shape={param.shape}, requires_grad={param.requires_grad}")
    
    # # Print model summary with torchinfo
    # print("\nModel Summary:")
    # # Assuming input image size of 224x224 with batch size 2
    # summary(model, input_data={"image": torch.zeros(2, 3, 224, 224)})


def freeze_backbone(model, freeze=True):
    """Freeze or unfreeze the backbone parameters"""
    for param in model.backbone.parameters():
        param.requires_grad = not freeze
    return model


def replace_regression_head(model, new_layers):
    """Replace the regression head with a new sequence of layers"""
    model.regression_head = nn.Sequential(*new_layers)
    return model


def change_output_dimension(model, output_dim):
    """Change the output dimension of the regression head"""
    # Create a new regression head with the specified output dimension
    new_head = nn.Sequential(
        nn.Linear(model.dim, output_dim),
        nn.ReLU(),
    )
    model.regression_head = new_head
    return model


In [15]:
# Calculate memory usage in MB
def get_model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    size_in_mb = (param_size + buffer_size) / 1024**2
    return size_in_mb

# Print the model size
model_size_mb = get_model_size(model=large_dinov)
print(f"Model size large: {model_size_mb:.2f} MB")

model_size_mb = get_model_size(model=small_dinov)
print(f"Model size small: {model_size_mb:.2f} MB")


NameError: name 'large_dinov' is not defined

In [12]:
print_model_info(model=model)

Total parameters: 1,136,488,449
Trainable parameters: 1,136,488,449
Frozen parameters: 0

Model Architecture:
DinoV2Finetune(
  (backbone): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1536, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-39): 40 x NestedTensorBlock(
        (norm1): LayerNorm((1536,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=1536, out_features=4608, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1536, out_features=1536, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((1536,), eps=1e-06, elementwise_affine=True)
        (mlp): SwiGLUFFNFused(
          (w12): Linear(in_features=1536, out_features=8192, bias=True)
          (w3): Linear(in_features=4096,

## VISUALIZE THE DATASET

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Load the dataset

train_data=pd.read_csv("../dataset/train_val.csv")


In [7]:
train_data.head(20)

,Unnamed: 0,id,channel,title,date,description,views,year
0,0,--2s6hjGrm4,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI & VFX Breakdowns: ""Warzone"" - by Ramesh Th...",2020-12-15 05:00:01+00:00,"Check out this revealing VFX Breakdown ""Warzon...",12299,2020
1,1,--DnfroyKQ8,UC-1rx8j9Ggp8mp4uD0ZdEIA,"A Sci-Fi Short Film: ""Exit"" - by Ng King Kwan ...",2020-07-01 16:00:00+00:00,"TheCGBros Presents ""Exit"" by Ng King Kwan - Th...",7494,2020
2,2,--aiU7VQKEw,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI 3D Animated Short: ""Lost Love"" - by Akash ...",2019-02-18 20:30:00+00:00,"TheCGBros Presents ""Lost Love"" by Akash Manack...",11831,2019
3,6,-0SrlZAvSVM,UCW6NyJ6oFLPTnx7iGRZXDDg,Jo Goes Hunting - Careful | Animated music vid...,2020-03-10 14:30:01+00:00,"On the borderless map of a magical planet, lit...",2248,2020
4,10,-13Y2Pe7kFs,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI VFX Breakdown: ""Logan (Wolverine): Digital...",2017-09-20 20:13:52+00:00,Check out this outstanding behind-the-scenes l...,113806,2017
5,12,-1ElNi3aewU,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI 3D Animated Spot : ""Aroma"" by - Space Patrol",2013-04-09 14:28:37+00:00,Here is another great 3D animated spot created...,6228,2013
6,13,-1FZlgerlnQ,UCXLfJIEp91L-fD0gP131IaQ,Hellevate Super Short Horror Comedy | Screamfest,2021-01-29 18:00:11+00:00,Hellevate is a super short horror comedy that ...,18652,2021
7,14,-1enK4XQvow,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI VFX Live Action Short Film: ""RELATIVITY"" ...",2022-01-05 17:00:00+00:00,"TheCGBros Presents ""RELATIVITY"" by Guillaume ...",20409,2022
8,15,-1hANFUO3yc,UC-1rx8j9Ggp8mp4uD0ZdEIA,"CGI VFX Short Films : ""TOB"" - by Andres Galeano",2014-03-18 00:31:30+00:00,"Watch this VFX short Film, by the talented And...",10370,2014
9,16,-1nkzRrnVfA,UCW6NyJ6oFLPTnx7iGRZXDDg,"Horror short film about a cursed cemetery | ""T...",2021-12-15 14:30:22+00:00,Sara and Ali are looking through photo's from ...,228067,2021


In [30]:
#load the train dataset
import torch
import torchvision.transforms as transforms
from data.dataset import Dataset

train_data_path="CSC_43M04_EP_challenge/dataset"
train_transformers=transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_set=Dataset(train_data_path, "train_val", train_transformers, metadata)

ModuleNotFoundError: No module named 'data'

In [26]:
import os
from PIL import Image
import matplotlib.pyplot as plt

# Define function to load and display an image
def load_and_display_image(image_id):
    image_path = os.path.join("dataset", "train_val", f"{image_id}.jpg")
    print(f"Loading image from: {image_path}")
    
    if os.path.exists(image_path):
        img = Image.open(image_path).convert("RGB")
        plt.figure(figsize=(10, 8))
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Image ID: {image_id}")
        plt.show()
        return img
    else:
        print(f"Error: Image file not found at {image_path}")
        return None

# Example: load the first image from your DataFrame
first_image_id = train_data['id'].iloc[0]  # Gets the ID of the first row
img = load_and_display_image(first_image_id)

Loading image from: dataset/train_val/--2s6hjGrm4.jpg
Error: Image file not found at dataset/train_val/--2s6hjGrm4.jpg


## TRY TO PERFORM INFERENCE ON THE TRAINED MODEL AND THE LARGEST MODEL

In [2]:
pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 51.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [transformers] [transformers]ub]
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader

# ----- Config -----
pretrained_model = "prajjwal1/bert-tiny"  # Very small model!
num_classes = 2
device = "cuda" if torch.cuda.is_available() else "cpu"

# ----- Dummy Dataset -----
texts = [
    "I love this movie!",     # pos
    "This was awesome",       # pos
    "I hate this film",       # neg
    "Terrible experience",    # neg
    "Brilliant performance",  # pos
    "Awful and boring"        # neg
]
labels = [1, 1, 0, 0, 1, 0]

class ToyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=32):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
        self.labels = torch.tensor(labels)
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

# ----- Custom Model -----
class MyCustomLLM(nn.Module):
    def __init__(self, base_model_path, num_classes):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(base_model_path)
        hidden_size = self.backbone.config.hidden_size
        self.classifier = nn.Linear(hidden_size, num_classes)
    def forward(self, input_ids, attention_mask):
        output = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        pooled = output.last_hidden_state[:, 0]  # CLS token
        return self.classifier(pooled)

# ----- Prepare -----
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
dataset = ToyDataset(texts, labels, tokenizer)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

model = MyCustomLLM(pretrained_model, num_classes).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# ----- Training Function -----
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for batch in loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# ----- Inference/Prediction -----
def predict(model, texts):
    model.eval()
    encoded = tokenizer(texts, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        logits = model(encoded['input_ids'], encoded['attention_mask'])
        probs = torch.softmax(logits, dim=1)
        preds = probs.argmax(dim=1)
    return preds.cpu().tolist(), probs.cpu().tolist()



In [3]:
# ----- Before Training -----
print("Predictions before training:")
preds, probs = predict(model, texts)
for t,p in zip(texts, preds):
    print(f"{t} -> {'positive' if p==1 else 'negative'}")

# ----- Training -----
for epoch in range(5):
    loss = train_one_epoch(model, loader, optimizer, criterion)
    print(f"Epoch {epoch+1} Loss: {loss:.4f}")

# ----- After Training -----
print("\nPredictions after training:")
preds, probs = predict(model, texts)
for t,p in zip(texts, preds):
    print(f"{t} -> {'positive' if p==1 else 'negative'}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Predictions before training:
I love this movie! -> negative
This was awesome -> positive
I hate this film -> negative
Terrible experience -> positive
Brilliant performance -> positive
Awful and boring -> negative
Epoch 1 Loss: 0.5483
Epoch 2 Loss: 0.6986
Epoch 3 Loss: 0.6109
Epoch 4 Loss: 0.5144
Epoch 5 Loss: 0.4636

Predictions after training:
I love this movie! -> positive
This was awesome -> positive
I hate this film -> negative
Terrible experience -> negative
Brilliant performance -> positive
Awful and boring -> negative
